In [1]:
import pandas as pd

# Lung Cancer Data Preprocessing
Source: https://wonder.cdc.gov/ucd-icd10.html
Origional Columns: `['Notes', 'County', 'County Code', 'Year', 'Year Code', 'Deaths',
       'Population', 'Crude Rate', 'Age Adjusted Rate']`

In [45]:
def cancer_preprocessing(df):
    """
    Preprocess cancer mortality data.

    Args:
        df (pd.DataFrame): Raw cancer data  

    Returns:
        (DataFrame): ['County', 'Year', 'Deaths', 'Crude Rate', 'Age Adjusted Rate']
    """

    # Some rows have "Unreliable"rate. Ignore them
    numeric_cols = ["Deaths", "Crude Rate", "Age Adjusted Rate"]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Reorder columns
    df = df[["County", "Year", "Deaths", "Crude Rate", "Age Adjusted Rate"]]
    df = df.dropna(subset=["County", "Year"], how="any")

    return df

In [47]:
# preprocess cancer data
ls_cancer_df = []

for i in range(1,4):
    # read in file
    file_path = f"cancer/death_{i}.csv"
    df_origional = pd.read_csv(file_path)
    
    # get out needed columns
    df_new = cancer_preprocessing(df_origional)
    
    ls_cancer_df.append(df_new)
    
df_cancer = pd.concat(ls_cancer_df, ignore_index=True)
df_cancer

,County,Year,Deaths,Crude Rate,Age Adjusted Rate
0,"Autauga County, AL",1999.0,32.0,74.5,84.4
1,"Autauga County, AL",2000.0,25.0,57.2,65.6
2,"Autauga County, AL",2001.0,28.0,62.4,73.2
3,"Autauga County, AL",2002.0,24.0,52.3,56.9
4,"Autauga County, AL",2003.0,20.0,42.7,46.5
...,...,...,...,...,...
46924,"Sweetwater County, WY",2016.0,10.0,NaN,NaN
46925,"Sweetwater County, WY",2017.0,19.0,NaN,NaN
46926,"Sweetwater County, WY",2018.0,16.0,NaN,NaN
46927,"Sweetwater County, WY",2019.0,13.0,NaN,NaN


In [48]:
# save to csv
df_cancer.to_csv("lung_cancer.csv")

# AQI Data Preprocessing
Source: https://aqs.epa.gov/aqsweb/airdata/download_files.html#Annual (AQI by County)<br>
Original Columns: `['State', 'County', 'Year', 'Days with AQI', 'Good Days',
       'Moderate Days', 'Unhealthy for Sensitive Groups Days',
       'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days', 'Max AQI',
       '90th Percentile AQI', 'Median AQI', 'Days CO', 'Days NO2',
       'Days Ozone', 'Days PM2.5', 'Days PM10']`<br>

In [28]:
STATE_TO_ABBR = {
        "Alabama": "AL",
        "Alaska": "AK",
        "Arizona": "AZ",
        "Arkansas": "AR",
        "California": "CA",
        "Colorado": "CO",
        "Connecticut": "CT",
        "Delaware": "DE",
        "District of Columbia": "DC",
        "Florida": "FL",
        "Georgia": "GA",
        "Hawaii": "HI",
        "Idaho": "ID",
        "Illinois": "IL",
        "Indiana": "IN",
        "Iowa": "IA",
        "Kansas": "KS",
        "Kentucky": "KY",
        "Louisiana": "LA",
        "Maine": "ME",
        "Maryland": "MD",
        "Massachusetts": "MA",
        "Michigan": "MI",
        "Minnesota": "MN",
        "Mississippi": "MS",
        "Missouri": "MO",
        "Montana": "MT",
        "Nebraska": "NE",
        "Nevada": "NV",
        "New Hampshire": "NH",
        "New Jersey": "NJ",
        "New Mexico": "NM",
        "New York": "NY",
        "North Carolina": "NC",
        "North Dakota": "ND",
        "Ohio": "OH",
        "Oklahoma": "OK",
        "Oregon": "OR",
        "Pennsylvania": "PA",
        "Rhode Island": "RI",
        "South Carolina": "SC",
        "South Dakota": "SD",
        "Tennessee": "TN",
        "Texas": "TX",
        "Utah": "UT",
        "Vermont": "VT",
        "Virginia": "VA",
        "Washington": "WA",
        "West Virginia": "WV",
        "Wisconsin": "WI",
        "Wyoming": "WY",
    }

def format_county(row):
    county = str(row["County"]).strip()
    abbr = str(row["State_abbr"]).strip()
    label = county + " County"

    return f"{label}, {abbr}"

In [30]:
def air_pollution_preprocessing(df):
    """
    Preprocess annual AQI data into wildfire-related features.

    Args:
        df (DataFrame): annual air pollution data (by county) from https://aqs.epa.gov/aqsweb/airdata/download_files.html#Annual
        year (int): year of the dataset

    Returns:
        (DataFrame): ['County', 'Year','Days_PM25_fraction','Unhealthy_fraction','VeryUnhealthy_fraction',
                      'Hazardous_fraction', 'Max_AQI', 'Median_AQI']
    """

    df = df.copy()
    
    # map state to abbreviation
    df["State_abbr"] = df["State"].map(STATE_TO_ABBR)

    # make sure target columns are numeric
    target_cols = [
        "Days with AQI",
        "Days PM2.5",
        "Unhealthy Days",
        "Very Unhealthy Days",
        "Hazardous Days",
        "Max AQI",
        "Median AQI",
    ]
    for col in target_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # avoid division by zero
    denom = df["Days with AQI"].replace(0, pd.NA)

    # get the fractions for wildfire-related columns
    df["Days_PM25_fraction"] = df["Days PM2.5"] / denom
    df["Unhealthy_fraction"] = df["Unhealthy Days"] / denom
    df["VeryUnhealthy_fraction"] = df["Very Unhealthy Days"] / denom
    df["Hazardous_fraction"]= df["Hazardous Days"] / denom

    # rename max & median AQI
    df["Max_AQI"] = df["Max AQI"]
    df["Median_AQI"] = df["Median AQI"]
    
    # reformat County -> "Los Angeles County, CA" so it could be used to combine with cancer data
    df["County"] = df.apply(format_county, axis=1)

    # select final columns
    out = df[["County", "Year", "Days_PM25_fraction", "Unhealthy_fraction", "VeryUnhealthy_fraction",
              "Hazardous_fraction", "Max_AQI", "Median_AQI"]].copy()

    return out


In [36]:
# preprocess all AQI data

# create a list to save all the dataframe
df_list = []

for year in range(1999,2021):
    # read in the csv
    file_path = f"AQI/annual_aqi_by_county_{year}.csv"
    df_original = pd.read_csv(file_path)
    
    # do the data pre-processing
    df_new = air_pollution_preprocessing(df_original)
    
    df_list.append(df_new)  

# concat all 21 dataframes
df_air_pollution = pd.concat(df_list, ignore_index=True)
df_air_pollution

,County,Year,Days_PM25_fraction,Unhealthy_fraction,VeryUnhealthy_fraction,Hazardous_fraction,Max_AQI,Median_AQI
0,"Clay County, AL",1999,0.234432,0.036630,0.0,0.000000,182,61
1,"Colbert County, AL",1999,1.000000,0.000000,0.0,0.000000,123,64
2,"DeKalb County, AL",1999,0.195783,0.057229,0.0,0.000000,195,61
3,"Elmore County, AL",1999,0.000000,0.004149,0.0,0.000000,151,45
4,"Escambia County, AL",1999,0.000000,0.000000,0.0,0.000000,52,20
...,...,...,...,...,...,...,...,...
23520,"Sublette County, WY",2020,0.060109,0.002732,0.0,0.000000,151,45
23521,"Sweetwater County, WY",2020,0.038251,0.010929,0.0,0.002732,1334,46
23522,"Teton County, WY",2020,0.139344,0.013661,0.0,0.000000,165,43
23523,"Uinta County, WY",2020,0.000000,0.000000,0.0,0.000000,122,40


In [37]:
# save to csv
df_air_pollution.to_csv("air_pollution.csv")

# Combine the Two Datasets
Combine based on County+Year

In [49]:
df_full = df_cancer.merge(df_air_pollution, on=["County", "Year"], how="inner")

In [50]:
print("Cancer rows:", len(df_cancer))
print("Air rows:", len(df_air_pollution))
print("Merged rows:", len(df_full))

Cancer rows: 46929
Air rows: 23525
Merged rows: 19213


In [51]:
# save to csv
df_full.to_csv("pollution_cancer.csv", ignore_index=True)